In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM , Dense , GlobalMaxPooling1D

from tensorflow.keras.layers import Input , Embedding

from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input, Embedding, GlobalMaxPooling1D, Dropout, Bidirectional, Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LSTM, Dense, Input, Embedding, GlobalMaxPooling1D, Dropout, Bidirectional, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, Dense, Input, Embedding, GlobalMaxPooling1D, Dropout
from tensorflow.keras.layers import LSTM, Dense, Input, Embedding, GlobalMaxPooling1D, Dropout, Bidirectional, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, Dense, Input, Embedding, GlobalMaxPooling1D, Dropout

from tensorflow.keras.callbacks import ModelCheckpoint


In [6]:
data = pd.read_csv('/content/drive/MyDrive/train.csv')
df = data
df = df.drop(['id','keyword','place'] , axis =  1)
df.columns = ['data','label']
y = df.iloc[:,1].values
x = df.iloc[:,0].values

x_train , x_test , y_train , y_test = train_test_split(x,y,test_size =0.2 , random_state = 42)

checkpoint_callback = ModelCheckpoint(filepath='/best_weights.keras',
                                          monitor='val_accuracy',
                                          verbose=1,
                                          save_best_only=True,
                                          mode='max')

MAX_VOCAB_SIZE = 20000
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)

# Fit tokenizer on training data
tokenizer.fit_on_texts(x_train)

# Convert text to sequences of word indices
sequences_train = tokenizer.texts_to_sequences(x_train)
sequences_test = tokenizer.texts_to_sequences(x_test)

data_train = pad_sequences(sequences_train)
print(data_train.shape)
T = data_train.shape[1]

data_test = pad_sequences(sequences_test , maxlen = T)
print(data_test.shape)


word2idx = tokenizer.word_index
v = len(word2idx)

D = 20
M = 15

# Define a callback to save the weights with the highest validation accuracy
checkpoint_callback = ModelCheckpoint(filepath='/best_weights.keras',
                                      monitor='val_accuracy',
                                      verbose=1,
                                      save_best_only=True,
                                      mode='max')


D = 20
M = 15

i = Input(shape = (T,))
x = Embedding(v +1 ,D)(i)
x = LSTM(128, return_sequences=True)(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1 , activation  = 'sigmoid')(x)
model = Model(i , x)

model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# Define early stopping callback

# Train the model
history = model.fit(
    data_train,
    y_train,
    epochs=20,  # Increase epochs to allow more training
    validation_data=(data_test, y_test),
    callbacks=[checkpoint_callback]
)

model.load_weights("/best_weights.keras")

# Evaluate the model on the test data
loss, accuracy = model.evaluate(data_test, y_test)

# Make predictions on the test data
y_pred = model.predict(data_test)

# Round the predictions to get binary values
y_pred_binary = (y_pred > 0.5).astype(int)


(6090, 33)
(1523, 33)
Epoch 1/20
190/191 [============================>.] - ETA: 0s - loss: 0.6195 - accuracy: 0.6470
Epoch 1: val_accuracy improved from -inf to 0.79842, saving model to /best_weights.keras
191/191 [==============================] - 15s 64ms/step - loss: 0.6192 - accuracy: 0.6473 - val_loss: 0.4690 - val_accuracy: 0.7984
Epoch 2/20
191/191 [==============================] - ETA: 0s - loss: 0.3582 - accuracy: 0.8548
Epoch 2: val_accuracy improved from 0.79842 to 0.80893, saving model to /best_weights.keras
191/191 [==============================] - 12s 61ms/step - loss: 0.3582 - accuracy: 0.8548 - val_loss: 0.4584 - val_accuracy: 0.8089
Epoch 3/20
191/191 [==============================] - ETA: 0s - loss: 0.2022 - accuracy: 0.9278
Epoch 3: val_accuracy did not improve from 0.80893
191/191 [==============================] - 12s 61ms/step - loss: 0.2022 - accuracy: 0.9278 - val_loss: 0.5295 - val_accuracy: 0.7905
Epoch 4/20
191/191 [==============================] - ETA: 

NameError: name 'f1_score' is not defined

In [7]:
from sklearn.metrics import f1_score, mean_squared_error

# Compute F1 score
f1 = f1_score(y_test, y_pred_binary)

# Compute mean squared error
mse = mean_squared_error(y_test, y_pred)

print("Test loss:", loss)
print("Test accuracy:", accuracy)
print("F1 score:", f1)
print("Mean squared error:", mse)

Test loss: 0.4584296941757202
Test accuracy: 0.8089297413825989
F1 score: 0.738544474393531
Mean squared error: 0.14499492652659252
